# EKF-SLAM using Factor Graphs (SRIF-like)

GTSAM Copyright 2010-2023, Georgia Tech Research Corporation,
Atlanta, Georgia 30332-0415
All Rights Reserved
Authors: Frank Dellaert, et al. (see THANKS for the full author list)

See LICENSE for the license information

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/python/examples/EKF_SLAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates 2D SLAM using GTSAM's factor graph tools in an **iterative, filter-like manner**, similar in structure to a Square Root Information Filter (SRIF) or an EKF, but leveraging factor graph elimination instead of explicit matrix operations.

**Scenario:**
A robot moves in a circular path within a 10x10 meter area containing randomly placed landmarks. It receives noisy odometry and bearing-range measurements.

**Factor Graph Filtering Approach:**
Instead of maintaining a large state vector and covariance matrix explicitly (like in classic EKF), we use GTSAM's `NonlinearFactorGraph` and `GaussianFactorGraph`:

1.  **Belief Representation:** The robot's belief (estimated state and uncertainty) after each step is implicitly represented by a factor graph (or the resulting Bayes Net after elimination).
2.  **Prediction:**
    *   Start with the factors representing the belief from the *previous* step.
    *   Add a new `BetweenFactorPose2` representing the noisy odometry measurement, connecting the previous pose `X(k)` to the new pose `X(k+1)`.
    *   Linearize all relevant factors around the current best estimate (`Values`).
    *   Eliminate the resulting `GaussianFactorGraph` using an ordering that marginalizes out the previous pose `X(k)`, yielding a `GaussianBayesNet` representing the *predictive* distribution over landmarks and `X(k+1)`.
3.  **Update:**
    *   Start with the factors representing the *predictive* belief.
    *   For each landmark measurement at the new pose `X(k+1)`:
        *   If the landmark `L(j)` is new, add it to the `Values` (initial estimate).
        *   Add a `BearingRangeFactor2D` connecting `X(k+1)` and `L(j)`.
    *   Linearize new/updated factors.
    *   Eliminate the `GaussianFactorGraph` to incorporate the measurement information, yielding a `GaussianBayesNet` representing the *posterior* belief.
    *   Optimize the final Bayes Net to get the updated state estimate (`Values`).
4.  **Iteration:** Repeat prediction and update for each time step.

**Enhancements in this version:**
*   Simulation code is moved to `simulation.py`.
*   Plotting code is moved to `gtsam_plotly.py`.
*   The evolving factor graph is displayed at each step using `graphviz`.

**Advantages:**
*   Leverages GTSAM's robust factor graph machinery.
*   Handles non-linearities through iterative linearization (like EKF, but potentially more robust within GTSAM's optimization context).
*   Avoids explicit management of large, dense covariance matrices.

**Output:** The process will be visualized with the evolving factor graph, and finally using a Plotly animation showing the robot's estimated path, mapped landmarks, and uncertainty ellipses evolving over time.

## 1. Setup and Imports

In [ ]:
# Install GTSAM and Plotly from pip if running in Google Colab
try:
    import google.colab
    %pip install --quiet gtsam-develop plotly
except ImportError:
    pass # Not in Colab

In [ ]:
import numpy as np
from tqdm.notebook import tqdm # Progress bar
import math
import time # To slow down graphviz display if needed

import gtsam
from gtsam.symbol_shorthand import X, L # Symbols for poses and landmarks

# Imports for new modules
import simulation
from gtsam_plotly import SlamFrameData, create_slam_animation

# Imports for graph visualization
import graphviz

## 2. Simulation Parameters

In [ ]:
# World parameters
NUM_LANDMARKS = 15
WORLD_SIZE = 10.0 # Environment bounds [-WORLD_SIZE/2, WORLD_SIZE/2]

# Robot parameters
ROBOT_RADIUS = 3.0
ROBOT_ANGULAR_VEL = np.deg2rad(20.0) # Radians per step
NUM_STEPS = 20 # Reduced steps for faster animation generation
DT = 1.0 # Time step duration (simplified)

# Noise parameters (GTSAM Noise Models)
# Prior noise on the first pose (x, y, theta)
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, np.deg2rad(1.0)]))
# Odometry noise (dx, dy, dtheta)
ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.05, np.deg2rad(2.0)]))
# Measurement noise (bearing, range)
MEASUREMENT_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([np.deg2rad(2.0), 0.2]))

# Sensor parameters
MAX_SENSOR_RANGE = 5.0

## 3. Generate Ground Truth Data

Use the `simulation` module to create true landmark positions, robot path, and simulate noisy measurements.

In [ ]:
landmarks_gt_dict, poses_gt, odometry_measurements, measurements_sim, landmarks_gt_array = \
    simulation.generate_simulation_data(
        num_landmarks=NUM_LANDMARKS,
        world_size=WORLD_SIZE,
        robot_radius=ROBOT_RADIUS,
        robot_angular_vel=ROBOT_ANGULAR_VEL,
        num_steps=NUM_STEPS,
        dt=DT,
        odometry_noise_model=ODOMETRY_NOISE,
        measurement_noise_model=MEASUREMENT_NOISE,
        max_sensor_range=MAX_SENSOR_RANGE,
        X=X, # Pass symbol functions
        L=L
    )

## 4. Iterative Factor Graph SLAM Implementation

Here we perform the step-by-step prediction and update using factor graphs. The evolving graph will be displayed.

In [ ]:
WRITER = gtsam.GraphvizFormatting()
WRITER.binaryEdges = True

def mage_dot(graph, estimate):
    return graph.dot(estimate, writer=WRITER)


In [ ]:
# --- Initialization ---
current_estimate = gtsam.Values() # Stores current best estimates (linearization points)
current_graph = gtsam.NonlinearFactorGraph() # Stores factors added so far
current_pose_key = X(0)

# Add prior on the initial pose X(0)
initial_pose = poses_gt[0] # Start at ground truth (or add noise)
# Add a bit of noise to the initial guess if desired:
# initial_pose_noisy = initial_pose.compose(gtsam.Pose2(0.1, 0.1, 0.01))
# current_estimate.insert(current_pose_key, initial_pose_noisy)
current_estimate.insert(current_pose_key, initial_pose) 

current_graph.add(gtsam.PriorFactorPose2(current_pose_key, initial_pose, PRIOR_NOISE))

# Variables to store results for animation
history  = [] # Store SLAMFrameData objects for each step
known_landmarks = set() # Set of landmark keys L(j) currently in the state

# Initial marginals (only for X(0))
initial_gaussian_graph = current_graph.linearize(current_estimate)
initial_marginals = gtsam.Marginals(initial_gaussian_graph, current_estimate)
history.append(SlamFrameData(0, current_estimate, initial_marginals, mage_dot(current_graph, current_estimate)))

### Main Iterative Loop

In [ ]:
print("Running Iterative Factor Graph SLAM...")

for k in tqdm(range(NUM_STEPS)):
    # === Prediction Step ===
    prev_pose_key = X(k)
    current_pose_key = X(k + 1)
    odom_k = odometry_measurements[k]
    
    # Predict next pose and add to Values (linearization point)
    # Ensure previous pose exists before accessing
    predicted_pose = current_estimate.atPose2(prev_pose_key).compose(odom_k)
    
    # Insert or update the current pose prediction
    current_estimate.insert_or_assign(current_pose_key, predicted_pose)
    
    # Add odometry factor to the graph
    current_graph.add(gtsam.BetweenFactorPose2(prev_pose_key, current_pose_key, odom_k, ODOMETRY_NOISE))

    # === Update Step ===
    measurements_k1 = measurements_sim[k + 1] # Measurements taken AT pose k+1
    landmarks_observed_this_step = set()

    for lm_key, measured_bearing, measured_range in measurements_k1:
        landmarks_observed_this_step.add(lm_key)
        
        # If landmark is new, initialize its estimate
        if not current_estimate.exists(lm_key):
            # Initial guess based on current pose estimate and measurement
            current_pose_val = current_estimate.atPose2(current_pose_key)
            delta_x = measured_range * math.cos(measured_bearing.theta())
            delta_y = measured_range * math.sin(measured_bearing.theta())
            # Transform delta from robot frame to world frame
            lm_initial_guess = current_pose_val.transformFrom(gtsam.Point2(delta_x, delta_y))
            current_estimate.insert(lm_key, lm_initial_guess)
            known_landmarks.add(lm_key)

        # Add measurement factor
        current_graph.add(gtsam.BearingRangeFactor2D(current_pose_key, lm_key, 
                                                        measured_bearing, measured_range,
                                                        MEASUREMENT_NOISE))
    
    # --- Optimization (Incremental Smoothing) ---
    optimizer = gtsam.LevenbergMarquardtOptimizer(current_graph, current_estimate)
    current_estimate = optimizer.optimize() # Update estimates by optimizing the whole graph so far
        
    # Calculate marginals for visualization (can be slow for large graphs)
    current_gaussian_graph = current_graph.linearize(current_estimate)
    current_marginals = gtsam.Marginals(current_gaussian_graph, current_estimate)

    # Store the current state for visualization
    history.append(SlamFrameData(k+1, current_estimate, current_marginals, mage_dot(current_graph, current_estimate)))

print("\nIterative Factor Graph SLAM finished.")
print(f"Final number of poses estimated: {len([key for key in current_estimate.keys() if gtsam.Symbol(key).chr() == ord('x')])}")
print(f"Number of landmarks mapped: {len(known_landmarks)}")

## 5. Create Plotly Animation

Use the `gtsam_plotly` module to visualize the results.

In [ ]:
fig = create_slam_animation(
    history,
    landmarks_gt_array=landmarks_gt_array,
    poses_gt=poses_gt,
    world_size=WORLD_SIZE,
    X=X,  # Pass symbol functions
    L=L,
    max_landmark_index=NUM_LANDMARKS,
)

print("Displaying animation...")
fig.show()

## 6. Discussion

*   **Approach:** This notebook implemented SLAM iteratively using GTSAM factor graphs. At each step, new factors (odometry, measurements) were added, and the graph was re-optimized using Levenberg-Marquardt. This is more akin to **incremental smoothing** than a classic filter (which would explicitly marginalize past states).
*   **Modularity:** Simulation and Plotly visualization code have been moved into separate `simulation.py` and `gtsam_plotly.py` files for better organization.
*   **Graph Visualization:** The `graphviz` library was used to render the factor graph at each step. This helps visualize how the graph structure grows and connects poses and landmarks.
*   **Efficiency:** Optimizing the entire graph at every step becomes computationally expensive for long trajectories. For real-time performance or large problems, **iSAM2 (Incremental Smoothing and Mapping)** is the preferred GTSAM algorithm. iSAM2 efficiently updates the solution by only re-linearizing and re-solving parts of the graph affected by new measurements.
*   **Accuracy vs. EKF:** This factor graph approach generally handles non-linearities better than a standard EKF because it re-linearizes during optimization. It avoids some of the consistency pitfalls associated with the EKF's single linearization point per step.
*   **Visualization:** The Plotly animation shows the evolution of the robot's path estimate, the map of landmarks, and their associated uncertainties (covariance ellipses). You can observe how measurements help refine the estimates and reduce uncertainty, especially when loops are closed (implicitly here through repeated observations of landmarks).